# Setup

In [1]:
import pyral as rl
import pandas as pds
import numpy as np
from IPython.display import display, Markdown

def markdown(txt):
    display(Markdown(txt))

server = "rally1.rallydev.com"
user = "przemyslaw.kadela@sabre.com"
workspace = "Sabre Production Workspace"
apikey = "_HD3HAAWlR4KiB63cDdlYzssaKrhSEEyv6N325XjKEw"


#project = "Naos (TDS-KRK)"   
#project = "OrderSvc-KRK-TEAM2-IFS-45387011"   
project = "Access Guardians"   

current_quarter = '2021Q1'
next_quarter = '2021Q2'

iteration_weights = [0.0, 12.0, 10.0, 4.0, 3.0, 1.0, 1.0]

# Establishing connection to Rally

In [2]:
rally = rl.Rally(server,user, apikey=apikey, workspace=workspace, project=project)

# Fetching iteration data

In [3]:
response = rally.get("Iteration", query=r'((StartDate >= "today-56") AND (StartDate < "today-14"))', fetch=False)
past_iterations = [d.Name for d in response]

response = rally.get("Iteration", query=r'StartDate > "today"', fetch=False)
future_iterations = [d.Name for d in response][0:6]

print("Past Iterations:", past_iterations)
print("Future Iterations:", future_iterations)

Past Iterations: ['S04#2021-02-10/02-23', 'S05#2021-02-24/03-09', 'S06#2021-03-10/03-23']
Future Iterations: ['S08#2021-04-07/04-20', 'S09#2021-04-21/05-04', 'S10#2021-05-05/05-18', 'S11#2021-05-19/06-01', 'S12#2021-06-02/06-15', 'S13#2021-06-16/06-29']


# Fetching user stories for current and next quarter under commited FEA

In [4]:
user_story_query = ( '('
    + f'((TeamFeature.Parent.ProductRoadmapQuarter = "{current_quarter}") OR (TeamFeature.Parent.ProductRoadmapQuarter = "{next_quarter}")) '
    + f'AND (TeamFeature.Parent.ProductRoadmapQuarterStatus = "Committed")' 
    + ')' )
print("Query:", user_story_query)

response = rally.get("User Story", query=user_story_query, fetch=True) 
user_story_data = [(x.FormattedID, x.Name, x.ScheduleState, x.PlanEstimate, x.Iteration.Name if x.Iteration else "-", "?") for x in response]

frm_user_stories = pds.DataFrame(user_story_data, columns=['Id', 'Title', 'State', 'Estimate', 'Iteration', 'Quarter'])
frm_user_stories

Query: (((TeamFeature.Parent.ProductRoadmapQuarter = "2021Q1") OR (TeamFeature.Parent.ProductRoadmapQuarter = "2021Q2")) AND (TeamFeature.Parent.ProductRoadmapQuarterStatus = "Committed"))


,Id,Title,State,Estimate,Iteration,Quarter
0,US92358,Perform Code Review clean up for ES-2.0,Idea,NaN,-,?
1,US92359,Add request type restrictions on MOM services,Idea,NaN,-,?
2,US92360,Create MOM Bootrstrap for metrics using Spring...,Idea,NaN,-,?
3,US92361,Implement more fine-grained logging using aspects,Idea,NaN,-,?
4,US92362,Set up uniform logging in Echo,Idea,NaN,-,?
...,...,...,...,...,...,...
284,US1105729,Clean packages in WAM CI/CD pipelines,Idea,NaN,S08#2021-04-07/04-20,?
285,US1105753,Install Pinger patching scripts in PROD,Idea,NaN,S08#2021-04-07/04-20,?
286,US1105914,Verify delegated admin capabilities in AM 6.0,In-Progress,5.0,S07#2021-03-24/04-06,?
287,US1111452,Introduce environment based regex to EmailBrid...,Idea,NaN,S08#2021-04-07/04-20,?


# Historical Data

In [5]:
frm_history = frm_user_stories[frm_user_stories["Iteration"].isin(past_iterations)]
hist_velocities = frm_history[["Iteration", "Estimate"]].groupby("Iteration").sum()
hist_throughput = frm_history[["Iteration", "Id"]].groupby("Iteration").count()

hist_average_velocity = np.mean(hist_velocities["Estimate"])
hist_average_throughput = np.mean(hist_throughput["Id"])
hist_us_size_median = np.median(frm_history["Estimate"])

metrics = {"history": {"average_velocity": hist_average_velocity, "average_throughput": hist_average_throughput, "us_size_median": hist_us_size_median}}

markdown("## History Data Metrics")
markdown("| Metric |   Value   |\n" + 
        "|-------------------:|:----------|\n" +
        f"|Average Velocity|{hist_average_velocity:.2f} |\n" +
        f"|Average Throughput|{hist_average_throughput:.2f} |\n" +
        f"|US size median|{hist_us_size_median:.2f} |" )

## History Data Metrics

| Metric |   Value   |
|-------------------:|:----------|
|Average Velocity|32.67 |
|Average Throughput|18.00 |
|US size median|2.00 |

# Backlog Metrics

In [7]:
frm_future = frm_user_stories[frm_user_stories["Iteration"].isin(future_iterations + ["-"])]
frm_iter_size = frm_future[["Iteration", "Estimate"]].groupby("Iteration").sum()
frm_iter_count = frm_future[["Iteration", "Id"]].groupby("Iteration").count()
frm_iter_size["CapacityIndex"] = frm_iter_size["Estimate"] / metrics["history"]["average_velocity"]
frm_iter_size["USCount"] = frm_iter_count["Id"]


backlog_sum = frm_future["Estimate"].sum()
backlog_count = frm_future["Id"].count()

backlog_index = backlog_sum / metrics["history"]["average_velocity"]
backlog_count_index = backlog_count / metrics["history"]["average_throughput"]
work_plan_index = frm_iter_size.query("Iteration != '-'")["CapacityIndex"].sum()

fc = frm_future[frm_future["Estimate"] <= metrics["history"]["us_size_median"]]
us_size_index = 2.0 * fc["Id"].count() / backlog_count

fi = future_iterations + ["-"]
frm_aux = pds.DataFrame([[ 0.0, 0.0, 0]] * len(fi), index=fi, columns=[ "Estimate", "CapacityIndex", "USCount"])
frm_aux = frm_iter_size.append(frm_aux).groupby(level=0).sum().sort_index()
frm_aux["weight"] = iteration_weights
frm_aux["value"] = frm_aux["weight"] * frm_aux["CapacityIndex"]

work_distribution_index = frm_aux["value"].sum() / frm_aux["weight"].sum()

markdown("### Metrics")
markdown("| Metric |   Expected   |   Value   |\n" + 
        "|-------------------:|:----------|:----------|\n" +
        f"|Backlog Index| 1.9 - 6.5 |**{backlog_index:.2f}** |\n" +
        f"|Backlog Count Index| 1.6 - 6.0 |**{backlog_count_index:.2f}** |\n" +
        f"|Work Plan Index| 1.9 - 6.5 |**{work_plan_index:.2f}** |\n" +
        f"|Work Distribution Index| 0.7 - 1.2 |**{work_distribution_index:.2f}** |\n" +
        f"|US Size Index| 0.7 - 1.1 |**{us_size_index:.2f}** |\n" +
        f"| - - - | - - - | - - - |\n" +
        f"|Backlog Count| - |{backlog_count:.0f} |\n" +
        f"|Backlog Size (SP)| - |{backlog_sum:.0f} |\n"  )


### Metrics

| Metric |   Expected   |   Value   |
|-------------------:|:----------|:----------|
|Backlog Index| 1.9 - 6.5 |**6.28** |
|Backlog Count Index| 1.6 - 6.0 |**8.00** |
|Work Plan Index| 1.9 - 6.5 |**3.43** |
|Work Distribution Index| 0.7 - 1.2 |**1.31** |
|US Size Index| 0.7 - 1.1 |**0.26** |
| - - - | - - - | - - - |
|Backlog Count| - |144 |
|Backlog Size (SP)| - |205 |
